In [1]:
!pip install -q "groq==0.11.0" requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.7 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from google.colab import userdata
import requests
import json
import os

In [3]:
Groq_api_key = userdata.get('GROQ_API_KEY')

In [4]:
from groq import Groq

client = Groq(api_key=Groq_api_key)

In [5]:
MODEL_NAME = "llama3-70b-8192"

In [6]:
weather_api_key = userdata.get('openweather')

In [7]:
API_key_Serach_engine = 'your_api_key_for_search_engine'
SEARCH_ENGINE_ID = 'search_engine_id'
searchType = 'image'

In [8]:
def fetch_image_results(query: str):
  url = f"https://www.googleapis.com/customsearch/v1?key={API_key_Serach_engine}&cx={SEARCH_ENGINE_ID}&q={query}&start=1&searchType={searchType}&imgSize=xlarge"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    search_items = data.get("items")
    image_url = search_items[1]['link']

    return image_url
  else:
    return None

In [9]:
def fetch_text_results(query: str):
    url = f"https://www.googleapis.com/customsearch/v1?key={API_key_Serach_engine}&cx={SEARCH_ENGINE_ID}&q={query}&start=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        search_items = data.get("items")
        snippet = search_items[1]['snippet']
        # return response.json().get("items", [])
        return snippet
    else:
        return None

In [10]:
def get_weather(city: str):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "city": city,
            "temperature": data['main']['temp'],
            "description": data['weather'][0]['description'],
            "humidity": data['main']['humidity'],
            "pressure": data['main']['pressure']
        }
        return weather
    else:
        return None

In [11]:
fetch_image_results("Jaffna")

'https://cf.bstatic.com/xdata/images/hotel/max1024x768/465172062.jpg?k=586f066619061d03ffa1bb69af5fdba9e8e13865fe6e50d82c22b5f83d17a007&o=&hp=1'

In [12]:
fetch_text_results("Large langugae models")

"LLMs can also be trained on other types of data, including code, images, audio, video, and more. Google's Codey, Imagen and Chirp are examples of such models\xa0..."

In [13]:
weather_function = {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },

                },
                "required": ["location"],
            },
        }
    }


In [14]:
fetch_text_results_function = {
    "type": "function",
    "function": {
        "name": "fetch_text_results",
        "description": "Fetch text search results from Google Custom Search API",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query string",
                },
                "start": {
                    "type": "integer",
                    "description": "The starting index of the results, for pagination",
                    "default": 1,
                },
            },
            "required": ["query"],
        },
    },
}


In [15]:
fetch_image_results_function = {
    "type": "function",
    "function": {
        "name": "fetch_image_results",
        "description": "Fetch image search results from Google Custom Search API",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query string",
                },
                "start": {
                    "type": "integer",
                    "description": "The starting index of the results, for pagination",
                    "default": 1,
                },
                "imgSize": {
                    "type": "string",
                    "description": "The size of images to fetch (e.g., small, medium, large, xlarge, etc.)",
                    "default": "large",
                },
            },
            "required": ["query"],
        },
    },
}


In [17]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

while True:
    user_input = str(input("Ask your question or If you want to stop it type as quit:- "))

    if user_input=='quit':
      break

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=[weather_function, fetch_text_results_function, fetch_image_results_function],
        tool_choice="auto",
        # max_tokens=4096,
    )

    tool_calls = response.choices[0].message.tool_calls

    if tool_calls:
        available_functions = {
            "get_weather": get_weather,
            "fetch_image_results": fetch_image_results,
            "fetch_text_results": fetch_text_results,
        }

        for tool_call in tool_calls:
            print(f"Function using: {tool_call.function.name}")
            # print(f"Params:{tool_call.function.arguments}")

            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]

            if function_name=="get_weather":
              function_args = json.loads(tool_call.function.arguments)
              function_response = function_to_call(
                  city=function_args.get("location"),
              )
            if function_name in ["fetch_text_results", "fetch_image_results"]:
              function_args = json.loads(tool_call.function.arguments)
              print(f"argumnets: {function_args}")
              function_response = function_to_call(
                  query=function_args.get("query"),
              )
            # print(f"API: {function_response}")

            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )

            second_response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
            )

            messages.append({
                "role": "assistant",
                "content": second_response.choices[0].message.content
            })

            print(second_response.choices[0].message.content)

    else:
        messages.append({
                "role": "assistant",
                "content": response.choices[0].message.content
            })

        print(response.choices[0].message.content)
    print("")

Ask your question or If you want to stop it type as quit:- best hotels in the Sri lanka
Function using: fetch_text_results
argumnets: {'query': 'best hotels in Sri Lanka'}
I apologize for the confusion! It seems like I misunderstood your request.

Here are some of the best hotels in Sri Lanka:

1. **Amanwella, Tangalle**: A luxurious beachfront resort with modern villas, private pools, and stunning ocean views.
2. **Wild Coast Tented Lodge, Yala**: An eco-friendly safari lodge with luxurious tented accommodations, outdoor pool, and breathtaking views of the Indian Ocean.
3. **Four Seasons Resort, Maldives**: A 5-star resort with luxurious villas, private pools, and stunning ocean views, perfect for snorkeling and diving.
4. **The Fortress, Koggala**: A luxurious beachfront hotel with modern rooms, outdoor pool, and stunning ocean views, perfect for water sports.
5. **Heritance Ahungalla, Ahungalla**: A luxurious beachfront hotel with modern rooms, outdoor pool, and stunning ocean views